In [7]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import MACD
from ta.volume import VolumeWeightedAveragePrice

# Define the ticker and date range
TICKER = 'INFY.NS'
START_DATE = '2019-01-01' # ~5 years of data
END_DATE = pd.to_datetime('today').strftime('%Y-%m-%d')

# Download data
df = yf.download(TICKER, start=START_DATE, end=END_DATE, auto_adjust=True)
df = df.dropna() # Drop any rows with missing data (e.g., trading holidays)

[*********************100%***********************]  1 of 1 completed


In [5]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=3d6d3f627fde7a780ebc26f58d1ea5461c0886baa2029e8feeec17b494f4c10c
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [8]:
#Target Variable Calculation (Monthly Growth Percentage)
# 'Adj Close' is the core feature for financial analysis
df['Close_Price'] = df['Adj Close']

# Target Variable (y): 21-day (approx. 1 month) Future Return Percentage
# Calculate the 'Close_Price' 21 trading days from now
df['Future_Close'] = df['Close_Price'].shift(-21)

# Calculate the 21-day growth percentage: (Future_Close - Current_Close) / Current_Close * 100
df['Target_Return'] = ((df['Future_Close'] - df['Close_Price']) / df['Close_Price']) * 100

# Drop the last 21 rows as their 'Target_Return' will be NaN (no future price)
df.dropna(subset=['Target_Return'], inplace=True)

# Display the head of the DataFrame with the new target
print("\nDataFrame with Target Variable:")
print(df[['Close_Price', 'Future_Close', 'Target_Return']].tail())

KeyError: 'Adj Close'

In [9]:
df.head()

Price,Close,High,Low,Open,Volume
Ticker,INFY.NS,INFY.NS,INFY.NS,INFY.NS,INFY.NS
Date,,,,,
2019-01-01,553.448486,554.488725,544.377637,550.036524,2943390
2019-01-02,556.777222,560.896576,550.951886,554.239050,7416655
2019-01-03,556.860413,563.393089,551.825618,559.232136,6827249
2019-01-04,550.119629,560.813309,541.756123,559.024079,7889310
2019-01-07,558.982483,560.563615,550.494128,553.406795,8046340
